Изменения:
1. balanced class_weight в логреге
2. удалил стоп слова
3. сделал sublinear_tf True

Изменение macro_f1 на валидации 0.1  
Изменение macro_f1 на тесте 0.06
  
Пробовал добавлять в обучающий датасет данные, но это привело только к ухудшению результата  
1. https://www.kaggle.com/datasets/yutkin/corpus-of-russian-news-articles-from-lenta (30000 новостей)
2. Датасет, 

In [ ]:
!pip install datasets
!python -m spacy download ru_core_news_sm

In [2]:
from typing import List, Tuple

In [3]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import spacy
import nltk
from datasets import load_dataset

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/egor/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
renames = {'Культура' : 'entertainment', 'Забота о себе' : 'health', 'Политика' : 'politics', 'Технологии и медиа' : 'science/technology',  'Спорт' : 'sports', 'Путешествия' : 'travel'}

def load_sib200_ru() -> Tuple[Tuple[List[str], List[int]], Tuple[List[str], List[int]], Tuple[List[str], List[int]], List[str]]:
    trainset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='train')
    temp = pd.read_csv('./news_articles.csv')[['title', 'category']]
    temp = temp[temp['category'].apply(lambda x: x in renames.keys())]
    temp['category'] = temp['category'].map(renames)
    X_train = trainset['text'] + list(temp['title'])
    y_train = trainset['category'] + list(temp['category'])
    valset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='validation')
    X_val = valset['text']
    y_val = valset['category']
    testset = load_dataset('Davlan/sib200', 'rus_Cyrl', split='test')
    X_test = testset['text']
    y_test = testset['category']
    categories = set(y_train)
    unknown_categories = set(y_val) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the validation set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    unknown_categories = set(y_test) - categories
    if len(unknown_categories) > 0:
        err_msg = f'The categories {unknown_categories} are represented in the test set, but they are not represented in the training set.'
        raise RuntimeError(err_msg)
    categories = sorted(list(categories))
    y_train = [categories.index(it) for it in y_train]
    y_val = [categories.index(it) for it in y_val]
    y_test = [categories.index(it) for it in y_test]
    return (X_train, y_train), (X_val, y_val), (X_test, y_test), categories

In [47]:
def normalize_text(s: str, nlp_pipeline: spacy.Language) -> str:
    stop_words = set(nltk.corpus.stopwords.words('russian'))
    doc = nlp_pipeline(s)
        
    lemmas = [('<NUM>' if token.like_num else token.lemma_.lower()) for token in filter(lambda it1: not it1.is_punct and str(it1) not in stop_words, doc) if token not in stop_words]
    if len(lemmas) == 0:
        return ''
    return ' '.join(lemmas)

In [48]:
train_data, val_data, test_data, classes_list = load_sib200_ru()

In [49]:
nlp = spacy.load('ru_core_news_sm')

In [50]:
class_probability = 1.0 / len(classes_list)
max_df = 1.0 - 0.2 * class_probability
print(f'Maximal document frequency of term is {max_df}.')

Maximal document frequency of term is 0.9714285714285714.


In [51]:
classifier = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1, sublinear_tf=True, ngram_range=(1,1))),
    ('cls', LogisticRegression(solver='liblinear', C=1000.0, penalty='l1', n_jobs=4, max_iter=200, random_state=42, class_weight='balanced'))
])

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_11505/2566799651.py:2: SyntaxWarning: invalid escape sequence '\w'
  ('vectorizer', TfidfVectorizer(token_pattern='\w+', max_df=max_df, min_df=1, sublinear_tf=True, ngram_range=(1,1))),


In [52]:
cv = GridSearchCV(
    estimator=classifier,
    param_grid={
        'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'cls__C': [1e-1, 1, 10, 100, 1000],
        'cls__penalty': ['l1', 'l2']
    },
    scoring='f1_macro',
    cv=5,
    refit=True,
    n_jobs=-1,
    verbose=True
)

In [53]:
cv.fit([normalize_text(it, nlp) for it in train_data[0]], train_data[1])

Fitting 5 folds for each of 30 candidates, totalling 150 fits


/home/egor/my-jupyter-env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(
/home/egor/my-jupyter-env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(
/home/egor/my-jupyter-env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(
/home/egor/my-jupyter-env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  warnings.warn(
/home/egor/my-jupyter-env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1271: UserWarning: 'n_jobs' > 1 does no

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.9714285714285714,
                                                        sublinear_tf=True,
                                                        token_pattern='\\w+')),
                                       ('cls',
                                        LogisticRegression(C=1000.0,
                                                           class_weight='balanced',
                                                           max_iter=200,
                                                           n_jobs=4,
                                                           penalty='l1',
                                                           random_state=42,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'cls__C': [0.1, 1, 10, 100, 1000],
                         'cls__penalty': ['l1', 'l2'],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]},
             scoring='f1_macro', verbose=True)

In [54]:
print('Best parameters:')
print(cv.best_params_)

Best parameters:
{'cls__C': 100, 'cls__penalty': 'l2', 'vectorizer__ngram_range': (1, 1)}


In [55]:
print('Best F1-macro:')
print(cv.best_score_)

Best F1-macro:
0.6715651528315998


In [56]:
print(f'Vocabulary size is {len(cv.best_estimator_.named_steps["vectorizer"].vocabulary_)}.')

Vocabulary size is 5348.


Изменения:
1. balanced class_weight в логреге
2. удалил стоп слова
3. сделал sublinear_tf True

In [43]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.88      0.78      0.82         9
         geography       0.64      0.88      0.74         8
            health       1.00      0.64      0.78        11
          politics       1.00      0.79      0.88        14
science/technology       0.62      0.84      0.71        25
            sports       0.89      0.67      0.76        12
            travel       0.58      0.55      0.56        20

          accuracy                           0.73        99
         macro avg       0.80      0.73      0.75        99
      weighted avg       0.76      0.73      0.73        99



In [44]:
y_pred = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.82      0.47      0.60        19
         geography       0.75      0.71      0.73        17
            health       0.64      0.64      0.64        22
          politics       0.76      0.83      0.79        30
science/technology       0.71      0.80      0.75        51
            sports       0.88      0.84      0.86        25
            travel       0.68      0.68      0.68        40

          accuracy                           0.73       204
         macro avg       0.75      0.71      0.72       204
      weighted avg       0.74      0.73      0.73       204



c данными с kaggle

In [28]:
y_pred = classifier.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.55      0.67      0.60         9
         geography       0.43      0.38      0.40         8
            health       0.50      0.18      0.27        11
          politics       0.35      0.79      0.49        14
science/technology       0.71      0.68      0.69        25
            sports       0.82      0.75      0.78        12
            travel       0.73      0.40      0.52        20

          accuracy                           0.57        99
         macro avg       0.58      0.55      0.54        99
      weighted avg       0.61      0.57      0.56        99



In [29]:
y_pred = classifier.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.43      0.53      0.48        19
         geography       0.57      0.24      0.33        17
            health       0.50      0.14      0.21        22
          politics       0.44      0.83      0.57        30
science/technology       0.57      0.57      0.57        51
            sports       0.77      0.80      0.78        25
            travel       0.79      0.68      0.73        40

          accuracy                           0.58       204
         macro avg       0.58      0.54      0.53       204
      weighted avg       0.60      0.58      0.56       204



С моими данными

In [57]:
y_pred = cv.predict([normalize_text(it, nlp) for it in val_data[0]])
print(classification_report(y_true=val_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.86      0.67      0.75         9
         geography       0.60      0.75      0.67         8
            health       1.00      0.55      0.71        11
          politics       0.69      0.79      0.73        14
science/technology       0.71      0.80      0.75        25
            sports       0.75      0.75      0.75        12
            travel       0.60      0.60      0.60        20

          accuracy                           0.71        99
         macro avg       0.74      0.70      0.71        99
      weighted avg       0.73      0.71      0.71        99



In [58]:
y_pred = cv.predict([normalize_text(it, nlp) for it in test_data[0]])
print(classification_report(y_true=test_data[1], y_pred=y_pred, target_names=classes_list))

                    precision    recall  f1-score   support

     entertainment       0.64      0.47      0.55        19
         geography       0.82      0.53      0.64        17
            health       0.75      0.68      0.71        22
          politics       0.78      0.83      0.81        30
science/technology       0.71      0.82      0.76        51
            sports       0.88      0.84      0.86        25
            travel       0.66      0.72      0.69        40

          accuracy                           0.74       204
         macro avg       0.75      0.70      0.72       204
      weighted avg       0.74      0.74      0.73       204

